In [1]:
import random
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import plotly.express as px
from sklearn.neighbors import KDTree

dataFolder = "Inpath"
dataset="buildingFull.xyz"
pcd = np.loadtxt(dataFolder+dataset)
xyz=pcd[:,:3]


tree = KDTree(np.array(xyz), leaf_size=2)  


nearest_dist, nearest_ind = tree.query(xyz, k=12)
np.mean(nearest_dist[:,1:],axis=0)

def ransac_plane(xyz, threshold=3, iterations=1000):
  inliers=[]
  equation = None
  n_points=len(xyz)
  i=1
  while i < iterations:
    idx_samples = random.sample(range(n_points), 3)
    pts = xyz[idx_samples]
    vecA = pts[1] - pts[0]
    vecB = pts[2] - pts[0]
    normal = np.cross(vecA, vecB)
    a,b,c = normal
    d=-np.sum(normal*pts[1])
    distance = (a * xyz[:,0] + b * xyz[:,1] + c * xyz[:,2] + d) / np.sqrt(a ** 2 + b ** 2 + c ** 2)
    idx_candidates = np.where(np.abs(distance) <= threshold)[0]
    
    if len(idx_candidates) > len(inliers):
      equation = [a,b,c,d]
      inliers = idx_candidates.copy()
    
    i+=1
  return equation, inliers


eq,idx_inliers=ransac_plane(xyz)
inliers=xyz[idx_inliers]


mask = np.ones(len(xyz), dtype=bool)
mask[idx_inliers] = False
outliers=xyz[mask]


result_folder="M:\\lidar\\Test\\"
np.savetxt(result_folder+dataset.split(".")[0]+"_inliers.xyz", inliers, fmt='%1.4f', delimiter=';')
np.savetxt(result_folder+dataset.split(".")[0]+"_outliers.xyz", outliers, fmt='%1.4f', delimiter=';')